In [13]:
from model import load_model
import cv2

In [14]:
test=load_model('VGG16')

In [15]:
model=test.model
layer_name='block5_conv3'

In [16]:
from explanation_models import gradcam

In [17]:
g= gradcam(model,layer_name)

In [18]:
import tensorflow as tf
orig_img=tf.keras.preprocessing.image.load_img('data/dog.jpg',target_size=(224, 224))
# img=cv2.resize(orig_img ,(224,224))
# cv2.imshow('dog',orig_img)

In [19]:
img=tf.keras.preprocessing.image.img_to_array(orig_img)

In [20]:
img = img.reshape((1, 224, 224, 3))

In [21]:
from tf.keras.applications.vgg16 import preprocess_input


ModuleNotFoundError: No module named &#39;tf&#39;